In [1]:
from matplotlib import pyplot as plt
import numpy as np
import json
import ipywidgets as widgets
import random

In [152]:
class Ant():
    def __init__(self): 
        self.pos = np.zeros((2,))
        # self.pos = np.ones((2,))
        
        # self.v = np.ones((2,))
        self.v = np.array((.0005, .0005))
        
        # self.v /= 500
        # self.v *= -1
        # self.pos *= -1
        
        
        self.has_food = False
        self.accl_scale = .0001
        
    def move(self):
    # ant checks for pheromones
        
        # print(self.pos)
        # assert(self.v < 1)
        self.pos += self.v
    
    ## return position for the query to be processed by pher. grid
    def check_phermones(self):
        return self.pos
    
    def check_food(self):
        return self.has_food
    
    def set_food(self):
        self.has_food = True
    
    def update(self, phermone):
        rng = np.random.default_rng()
        accl = 2 * rng.random((2, 1)) -1
        # accl = np.random.randn(2, 1)
        accl /= np.linalg.norm(accl, axis=0)
        accl  *= self.accl_scale
        # print("phermone",phermone)
        # print(np.linalg.norm(phermone, axis=0))
        # print("phermone/np.linalg.norm(phermone)", phermone/np.linalg.norm(phermone, axis=0))
        # print(np.linalg.norm(phermone/np.linalg.norm(phermone, axis=0)))
        # print('\n\n')
        # print("phermone",phermone)
        # print(np.squeeze(accl))/
        # phermone /= np.linalg.norm(phermone, axis=0)
        
        # print('self.v:',self.v)
        
        self.v += np.squeeze(accl)
        # self.v += np.squeeze(phermone)
        
        # self.v += phermone
        
        # print('self.v:',self.v)
        # self.v /= np.linalg.norm(self.v, axis=0)
        # self.v *= self.accl_scale
        # print('np.linalg.norm(self.v, axis=0)',np.linalg.norm(self.v, axis=0))
        
        # print('self.v:',self.v, '\n\n')
        self.move()
        


In [161]:
class Grid:
    
    # def __init__(steps, xdim, ydim, decay, grid, size, phermone_grid, config_path, self):
    def __init__(self, steps, xdim, ydim, decay, scale, phermone_granularity):
    
        self.steps = steps
        self.xdim = xdim 
        self.ydim = ydim 
        self.decay = decay
        self.size = scale
        # self.grid = np.zeros((self.ydim, 
        #                       self.xdim,
        #                      3
        #                      ))
        
        # self.grid[0] == x indicies
        self.x_inds = np.linspace(-1,1,self.xdim-1)
        self.y_inds = np.linspace(-1,1,self.ydim-1)
        
        # self.grid = np.meshgrid(self.x_inds, self.y_inds)
        
        self.position_map = np.zeros((self.ydim, self.xdim, 1))
        # self.phermone_map = np.zeros((self.ydim, self.xdim, 2))
        self.phermone_map_to_food = np.zeros((self.ydim, self.xdim, 2))
        self.phermone_map_to_home = np.zeros((self.ydim, self.xdim, 2))
        self.food_map = np.zeros((self.ydim, self.xdim, 1))
        self.food_map[int(self.ydim/8):, :int(self.ydim/8)] = True
        
        
        
#         print('self.x_inds.shape', self.x_inds.shape)
#         print('pos map shape',  self.position_map.shape)
#         print("self.phermone_map_to_home[20,23]", self.phermone_map_to_home[20,23])
        
    # Given a continous position within (-1, 1), return the indicies of the grid cell containing the position.
    
    def pos_to_pixel(self, pos):
        
        if pos[0] > 1:
            pos[0] -= 2
        elif pos[0] < -1:
            pos[0] += 2
        elif pos[1] > 1:
            pos[1] -= 2
        elif pos[1] < -1:
            pos[1] += 2

        x_ind = np.digitize(pos[1], self.x_inds)
        y_ind = np.digitize(pos[0], self.y_inds)
        
        # print('got ant at:', pos)
        # print('converted position to:', x_ind, y_ind)

        return (y_ind, x_ind)
    
    def draw(self, ant, val):
        x, y = self.pos_to_pixel(ant.pos)
        x *= -1
        # print("ant.v", ant.v)

        if ant.has_food:
            self.phermone_map_to_food[x,y] += -1*ant.v  
        else:
            self.phermone_map_to_home[x,y] += -1*ant.v  
            # print("self.phermone_map_to_home[x,y]", self.phermone_map_to_home[x,y], '\n\n')
            
            
           
        self.position_map[x,y] = val  
        
        

        
        
    ### TODO Implement 
    def decayfunc(self):
        
        x_inds, y_inds, _ = np.nonzero(self.position_map)
        self.position_map[x_inds, y_inds]  -=  (self.decay) / 255
        self.position_map = np.where(self.position_map < 0.0, 0.0, self.position_map)
        
        #Phermones are 2-D
#         x_inds, y_inds, _ = np.nonzero(self.phermone_map_to_home)
#         self.phermone_map_to_home[x_inds, y_inds]  -=  (self.decay/5) / 255
#         # self.phermone_map_to_home = np.where(self.phermone_map_to_home < 0.0, 0.0, self.phermone_map_to_home)
#         # self.phermone_map_to_home = np.where(self.phermone_map_to_home > 1.0, 1.0, self.phermone_map_to_home)
        
        
#         x_inds, y_inds, _ = np.nonzero(self.phermone_map_to_food)
#         self.phermone_map_to_food[x_inds, y_inds]  -=  (self.decay/5) / 255
        # self.phermone_map_to_food = np.where(self.phermone_map_to_food < 0.0, 0.0, self.phermone_map_to_food)
        # self.phermone_map_to_food = np.where(self.phermone_map_to_food > 1.0, 1.0, self.phermone_map_to_food)
        
        # x_inds, y_inds, _ = np.nonzero(self.phermone_map)
        # self.phermone_map[x_inds, y_inds]  -=  (self.decay * .001) / 255
        # self.phermone_map = np.where(self.phermone_map < 0.0, 0.0, self.phermone_map)
        # self.phermone_map = np.where(self.phermone_map > 1.0, 1.0, self.phermone_map)
        
        

    def get_phermone(self, ant):
        
        x, y = self.pos_to_pixel(ant.pos)
        v_phermone = copy.deepcopy(self.phermone_map_to_food[x, y])
        # if np.any(v_phermone):
        #     print('phermone at', x, y, ':', v_phermone)
            
        

        # print("v_phermone",v_phermone)
#         if self.phermone_map_to_food[x, y]:
#             v_phermone = self.phermone_map_to_food[x, y]

#             print("v_phermone",v_phermone)
            
#         v_phermone = self.phermone_map_to_food[x, y]
        
        if self.food_map[x,y]:
            # print("found food")
            ant.set_food()
            
        if ant.has_food:
            v_phermone = self.phermone_map_to_home[x, y]

        return v_phermone

    def binarize_phermone_map(self, ph_map):
        
        # agg = np.sum(ph_map, axis=-1)
        agg = 10 * np.linalg.norm(ph_map, axis=-1)
        # print(agg[50,50])
        # print(agg[:,:,np.newaxis].shape)
        
        return agg[:,:,np.newaxis]
        

In [162]:

import copy
def simulate(grid, colony): 
    sim_accum = []
    food_accum = []
    home_accum = []
    

    for t in range(grid.steps):
        for a in colony:
          
            # check for phermone vector
            v_phermone=grid.get_phermone(a)
            # print(a.v)
            # print("in main:",v_phermone)
            a.update(v_phermone)
            grid.draw(a, 1)
            


        # We should save both phermone maps so we can watch the directions evolve over time using a quiver plot
        home_accum.append(grid.phermone_map_to_home)
        food_accum.append(grid.phermone_map_to_food)
        
        
        # Here we will binarize both phermone maps by aggregating their component vectors # This is done purely for visualization purposes. 
        # print('grid.phermone_map_to_home.shape',grid.phermone_map_to_home.shape)
        home_agg = grid.binarize_phermone_map(grid.phermone_map_to_home)
        food_agg = grid.binarize_phermone_map(grid.phermone_map_to_food)

        
        # Here we will stack the position and phermone maps as color channel
        frame = np.concatenate((copy.deepcopy(grid.position_map), food_agg, home_agg),axis=-1)
        sim_accum.append(frame)
        
        grid.decayfunc()

    sim_accum = np.array(sim_accum)
    return sim_accum


In [ ]:
%matplotlib widget
# %matplotlib auto
# g = Grid('config.json')
f = open('config.json', "rb")
config = json.load(f)

g = Grid(ydim= config['ydim'], 
         xdim= config['xdim'],
        decay=config['decay'],
         scale=config['scale'],
         steps=config['steps'],
         phermone_granularity=config['phermone_granularity'])


colony = []
for i in range(0, 1000):
    colony.append(Ant())

frames = simulate(g, colony)
print(frames[0].shape)

def update(change):
    im.set_data(frames[change['new']])
    fig.canvas.draw_idle()
    
    
with plt.ioff():
    fig = plt.figure()
    print(frames[0].shape)
im = plt.imshow(frames[0])
slider = widgets.IntSlider(value=0, min=0, max=frames.shape[0]-1)
slider.observe(update, names='value')
widgets.VBox([slider, fig.canvas])

In [ ]:
%matplotlib widget

with plt.ioff():
    fig = plt.figure()
    print(frames[0].shape)
im = plt.imshow(frames[0])
slider = widgets.IntSlider(value=0, min=0, max=frames.shape[0]-1)
slider.observe(update, names='value')
widgets.VBox([slider, fig.canvas])

In [7]:
# %timeit simulate(g, colony)

In [8]:
# always have x and y coordinates within -1<x<1
# coordinates represent 
# then conversion to pixels is simple 

In [9]:
# pheromones is array of velocity vectors. This array may be smaller than position grid and should have dimensions that are factors of corresponding position grid.
# should support the following functions: 
#     ant position (float x, y between -1, 1) -> phermone vector

In [10]:
# for each ant:
    # check phermone map at current pos
    # if phemrone, use phermone to update position 
    # else update position randomly
    # update phermone map

In [108]:
#data structures

# grids

##ant positions
# pos 

## 
# food_str 
# home_str

# food_data
# home_data
np.random.randn(2, 1)

array([[-0.77314862],
       [-0.27549747]])

In [109]:
np.random.rand(2)

array([0.74057393, 0.91010888])

In [125]:
# Three-by-two array of random numbers from [-5, 0):


array([[-0.56266367],
       [ 0.64120179]])